In [ ]:
# Descomenta esta linea para instalar los paquetes
%pip install -q TPOT==0.11.7 scikit-learn==0.24.2

<center>
<img src='mioti_logo.jpeg' height='80px' />
<p style="font-size: 18px"><b>Machine learning 3</b><br/>Sergio González Martín - sergiogm@faculty.mioti.es</p>
</center>
<br/>

# Worksheet S1: Hiperparametrización

# Objetivos

El objetivo de este worksheet es que aprendas a hiperparametrizar algoritmos conociendo sus ventajas, inconvenientes y aspectos prácticos necesarios para hacerlo.

## Prerrequisitos

Para hacer funcionar este worksheet deberás tener instalados los paquetes:

* `numpy`
* `pandas`
* `scikit-learn`
* `tpot`

## Configuración del entorno

In [ ]:
%matplotlib inline

import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split

También vamos a cargar algunas funciones que nos resultarán interesantes.

In [ ]:
def dibuja_dataset(X, y):
    from matplotlib.colors import ListedColormap

    cm_bright = ListedColormap(["#00FF00", "#0000FF"])
    plt.figure(figsize=(16, 8))
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright, edgecolors="k")

# Dataset

Para empezar vamos a generar un dataset ficticio:

In [ ]:
from sklearn.datasets import make_moons

dataset_X, dataset_y = make_moons(noise=0.7, random_state=10)

In [ ]:
dibuja_dataset(dataset_X, dataset_y)

In [ ]:
# Dividimos en conjunto de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_y, test_size=0.3, random_state=42)

# ¿Qué define un modelo?

- Estructura: el algoritmo matemático o arquitectura.

- Parámetros:
    - Parametros internos o intrínsecos: también llamados comunmente "pesos", son aquellos que se aprenden a través del entrenamiento en base al dataset escogido
    - Parametros externos o extrínsecos (hiperparámetros): no son parámetros entrenables, sino elegidos de forma 'manual', podemos pensar que cuando hablamos de un modelo (por ejemplo, una regresión logística) en realidad lo que tenemos es una familia de modelos en función de sus hiperparámetros
    

# Hiperparametrización

Cada modelo que utilicemos dispone de un montón de parámetros que determinan el comportamiento del mismo. Una buena o mala parametrización puede suponer la diferencia entre un modelo bien o mal entrenado.

Además sabemos que el nivel de complejidad del modelo tiene que ser capaz de ser suficiente para modelar la complejidad de los datos (http://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html). Sin embargo, si generamos un modelo mucho más complejo que los datos podemos incurrir en el problema del sobreajuste:

In [ ]:
from IPython.display import Image

Image(filename="complejidad_vs_sobreajuste.png", width="50%")

## ¿Cómo encontrar ese punto ideal?

No existe una respuesta "mágica" para esta pregunta, el método más habitual consiste en probar distintas configuraciones, de una forma más o menos inteligente, y seleccionar la que mejor ajusta.

Cuando queremos modificar un sólo parámetro `sklearn` proporciona una función que nos permite dibujar una curva de validación que pone en correspondencia los valores obtenidos de entrenamiento y los test. Por defecto esta función internamente realiza una validación cruzada para separar entre conjunto de entrenamiento y test.

Podemos ver como se comporta con un modelo de KNN variando el número de vecinos:

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.neighbors import KNeighborsClassifier

k_vecinos = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 31, 41, 50, 65]

train_scores, test_scores = validation_curve(
    KNeighborsClassifier(), dataset_X, dataset_y, param_name="n_neighbors", param_range=k_vecinos
)
plt.figure(figsize=(16, 8))
plt.plot(k_vecinos, train_scores.mean(axis=1), "b", label="Entrenamiento")
plt.plot(k_vecinos, test_scores.mean(axis=1), "g", label="Test")
plt.ylabel("Precisión")
plt.xlabel("# vecinos")
plt.legend(loc="best");

Y variando el valor C en una SVM:

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.svm import SVC

C_values = [0.001, 0.005, 0.01, 0.015, 0.2, 0.5, 1.0, 2.0]

train_scores, test_scores = validation_curve(SVC(), dataset_X, dataset_y, param_name="C", param_range=C_values)
plt.figure(figsize=(16, 8))
plt.plot(C_values, train_scores.mean(axis=1), "b", label="Entrenamiento")
plt.plot(C_values, test_scores.mean(axis=1), "g", label="Test")
plt.ylabel("Precisión")
plt.xlabel("C")
plt.legend(loc="best");

Si queremos modificar más de un parámetro a la vez deberemos construirnos nosotros el bucle de evaluación de los parámetros:

In [ ]:
from sklearn.svm import SVC

# Probamos la combinación de parámetros
for kernel in ["linear", "rbf", "sigmoid"]:
    for C in [0.001, 0.005, 0.0075, 0.01, 0.015, 0.02, 0.1, 1, 10]:
        scores = cross_val_score(SVC(kernel=kernel, C=C, random_state=42), dataset_X, dataset_y)
        print("Kernel: %8s\tC: %3.3f\tAccuracy: %.3f" % (kernel, C, np.mean(scores)))

## GridSearch

Alternativamente a implementarlo manualmente (que tiene sus ventajas e inconvenientes) `sklearn` proporciona funciones para hacer este tipo de cosas como la función `GridSearchCV`. Esta función realiza nuestro trabajo por nosotros y además provee funcionalidad adicional como cálculo en paralelo.

Como puedes ver su uso es bastante sencillo:

In [ ]:
from sklearn.model_selection import GridSearchCV

params_grid = {
    "C": [0.001, 1, 10],
    "kernel": ["linear", "rbf", "sigmoid"],
    "random_state": [42],
}

grid_svm = GridSearchCV(SVC(), param_grid=params_grid, n_jobs=4, scoring="roc_auc")

Esta función es interesante por que es un *meta-estimador* es decir se utiliza exactamente igual que un estimador cualquiera:

In [ ]:
grid_svm.fit(X_train, y_train)

Podemos consultar cual es el mejor resultado obtenido:

In [ ]:
print(grid_svm.best_score_)

Y con que parámetros:

In [ ]:
print(grid_svm.best_params_)

Y podemos consultar con detalle todos las evaluaciones realizadas y los resultados obtenidos:

In [ ]:
pd.DataFrame(grid_svm.cv_results_).head()

## RandomizedSearchCV

La función `GridSearchCV` realiza una búsqueda exhaustiva en el espacio de solución. Muchas veces debido a la explosión combinatoria al comnbinar parámetros esto no es posible. En estas situaciones lo que se suele hacer es realizar un muestreo del espacio de soluciones. Uno de los algoritmos más sencillos y utilizados es el muestreo aleatorio.

En `sklearn` lo podemos aplicar con la función `RandomizedSearchCV`, su uso es parecido a `GridSearchCV` con la principal diferencia de que en vez de proporcionar un diccionario con los posibles parámetros debemos proporcionar uno con las distribuciones de los parámetros. Para especificarlas es cómodo utilizar las funciones para especificar distribuciones que nos proporciona `scipy`:

In [ ]:
import scipy.stats as st

a = st.uniform(0.01, 10)

In [ ]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV

params_dist = {"C": st.uniform(0.01, 10), "kernel": ["linear", "rbf", "sigmoid"], "random_state": [42]}

random_svm = RandomizedSearchCV(
    SVC(), param_distributions=params_dist, n_jobs=4, n_iter=100, scoring="roc_auc", random_state=42
)

In [ ]:
random_svm.fit(X_train, y_train)

In [ ]:
print(random_svm.best_params_)

In [ ]:
print(random_svm.best_score_)

# Métodos Bayesianos

Utilizan el teorema de Bayes para optimizar los hiperparámetros, el proceso es el siguiente:
- Definir un 'prior': una inicialización inicial para tu función objetivo
- Evaluar en unos puntos iniciales
- En base al resultado anterior y una función de adquisición, elegir cual es el punto más 'prometedor' para evaluar a continuación (exploitation vs exploration)
- Repeat

<center>
<img src='bayesian_optimization.png' height='480px' />

</center>
<br/>

Algunas librerías que lo implementan en python son: [BayesianOptimization](https://github.com/bayesian-optimization/BayesianOptimization), [hyperopt](https://github.com/hyperopt/hyperopt/tree/master), [scikit-optimize](https://github.com/scikit-optimize/scikit-optimize)



## Errores al hiperparametrizar

Como hemos visto, la hiperparametrización es un mecanismo efectivo en muchos casos para mejorar el rendimiento de un modelo. Pero.. ¿cuando hiperparametrizamos no caemos en la posibilidad de ajustar demasiado los parámetros a un conjunto de entrenamiento?. **¿Son los hiperparámetros parte o no del modelo?**.

<center style="color:#888; margin: 50px 0 50px 0">...</center>

La respuesta es que sí, los hiperparámetros son también parte del modelo (quizás de orden distinto a los valores internos de los modelos en si).

Para evitar los errores de la hiperparametrización se suele realizar una validación de la hiperparametrización. Una estrategia de entrenamiento muy frecuente es la siguiente:

<br/>
<center><img src='grid_search_cv.png' width='50%'></center>

Podemos aplicar esta estrategia de la siguiente manera:

Podemos aplicar esta estrategia de la siguiente manera:

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_y, test_size=0.3, random_state=42)


# Ajustamos los hiperparámetros a analizar
params_grid = {"C": [0.001, 0.01, 0.1, 1, 10], "kernel": ["linear", "rbf", "sigmoid"], "random_state": [42]}

grid_svm = GridSearchCV(SVC(), param_grid=params_grid)

# Entrenamos el modelo
grid_svm.fit(X_train, y_train)

# Lo evaluamos
grid_svm.score(X_test, y_test)

## Quizz

* ¿Hiperparametrización si o no?
* ¿Qué riesgos incurrimos si hiperparametrizamos? ¿cómo podemos mitigarlos? 
* ¿Qué riesgos incurrimos si no hiperparametrizamos?

# Hiperparametrización automática

Una de las áreas de mayor desarrollo en machine learning en los últimos años ha sido el desarrollo de algoritmos y servicios de hiperparametrización automática de modelos.

Al margen de los servicios cloud estilo AutoML de Google, SageMaker de Amazon... en este worksheet vamos a trabajar con TPOT (http://epistasislab.github.io/tpot/) que es un optimizador de modelos de sklearn basado en algoritmos genéticos:

TPOT intenta automatizar los procesos de:
* Preprocesamiento y construcción de features
* Selección y optimización de modelos


<center><img src='tpot-ml-pipeline.png' width='65%'></center>

Para ello aplica técnicas como:

* Preprocesamiento: distintas estrategias de valores ausentes, normalización de los datos, análisis de componentes principales (PCA), selección de atributos
* Evaluar distintos modelos con distintos valores de hiperparámetros.
* Combinación de modelos mediante técnicas de ensemble.

Como podemos ver a continuación su uso es muy sencillo, aunque debemos disponer de tiempo y potencia de cálculo para obtener resultados:

In [ ]:
from tpot import TPOTClassifier

In [ ]:
pipeline_optimizer = TPOTClassifier(
    generations=3,
    population_size=20,
    cv=5,
    random_state=41,
    max_time_mins=5,
    scoring="roc_auc",
    verbosity=2,
    n_jobs=4,
)

Parámetros interesantes son:

* `generations`: número de iteraciones completas que se optimiza el algoritmo
* `population_size`: tamaño de la población a evaluar en cada generación. Por ejemplo si configuramos 5 generaciones con un tamaño de población de 20, se evaluarán 5*20=100 modelos distintos.
* `scoring`: define la métrica de evaluación del algoritmo. Están soportadas las siguientes opciones: 'accuracy', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'neg_log_loss','precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc' 
* `max_time_mins`: define el tiempo máximo, expresado en minutos, de ejecución del algoritmo.

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

Una vez construido el modelo podemos evaluarlo:

In [ ]:
print(pipeline_optimizer.score(X_test, y_test))

Y también exportarlo en un fichero independiente:

In [ ]:
pipeline_optimizer.export("tpot_exported_pipeline.py")

## Control de la ejecución

Algo interesante que tiene `tpot` es que podemos parar en cualquier momento la ejecución con `ctrl+c` si estamos en una terminal o dando al botón _stop_ en jupyter. Esto parará la ejecución y nos mostrará el mejor resultado hasta el momento. Con el parámetro `warm_start` podemos hacer incluso que si reejecutamos la función `fit` el algoritmo continue por donde estaba.

In [ ]:
pipeline_optimizer = TPOTClassifier(
    generations=5,
    population_size=20,
    cv=5,
    random_state=42,
    verbosity=2,
    max_time_mins=3,
    scoring="roc_auc",
    warm_start=True,
)

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

Como verás las ejecuciones tardan bastante, es normal. Estos sistemas están diseñados para ser ejecutados durante horas incluso días.

Si tienes más tiempo, puedes ejecutar el siguiente ejemplo con el dataset `digits` y ver como consigue casi un 99% de accuracy:

In [ ]:
print(pipeline_optimizer.score(X_test, y_test))

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)

In [ ]:
pipeline_optimizer = TPOTClassifier(
    generations=15,
    population_size=20,
    cv=5,
    random_state=42,
    verbosity=2,
    n_jobs=12,
)

pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export("tpot_exported_pipeline.py")

In [ ]:
pipeline_optimizer.export("tpot_exported_pipeline.py")

## Quizz

* ¿Hiperparametrización automática si o no?
* ¿En qué casos nos interesa una hiperparametrización manual o automática?